In [2]:
import numba as nb
import numpy as np
import scipy.spatial as spat
import os 
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from datetime import datetime
from datetime import timedelta
from scipy import optimize
from matplotlib import pyplot as plt, cm, colors

In [3]:
#function for extracting faces of alpha shape of point cloud
"""
https://github.com/timkittel/alpha-shapes

Copyright 2018, Tim Kittel

Redistribution and use in source and binary forms, with or without modification, are permitted provided that the following conditions are met:

1. Redistributions of source code must retain the above copyright notice, this list of conditions and the following disclaimer.

2. Redistributions in binary form must reproduce the above copyright notice, this list of conditions and the following disclaimer in the documentation and/or other materials provided with the distribution.

3. Neither the name of the copyright holder nor the names of its contributors may be used to endorse or promote products derived from this software without specific prior written permission.

THIS SOFTWARE IS PROVIDED BY THE COPYRIGHT HOLDERS AND CONTRIBUTORS "AS IS" AND ANY EXPRESS OR IMPLIED WARRANTIES, INCLUDING, BUT NOT LIMITED TO, THE IMPLIED WARRANTIES OF MERCHANTABILITY AND FITNESS FOR A PARTICULAR PURPOSE ARE DISCLAIMED. IN NO EVENT SHALL THE COPYRIGHT HOLDER OR CONTRIBUTORS BE LIABLE FOR ANY DIRECT, INDIRECT, INCIDENTAL, SPECIAL, EXEMPLARY, OR CONSEQUENTIAL DAMAGES (INCLUDING, BUT NOT LIMITED TO, PROCUREMENT OF SUBSTITUTE GOODS OR SERVICES; LOSS OF USE, DATA, OR PROFITS; OR BUSINESS INTERRUPTION) HOWEVER CAUSED AND ON ANY THEORY OF LIABILITY, WHETHER IN CONTRACT, STRICT LIABILITY, OR TORT (INCLUDING NEGLIGENCE OR OTHERWISE) ARISING IN ANY WAY OUT OF THE USE OF THIS SOFTWARE, EVEN IF ADVISED OF THE POSSIBILITY OF SUCH DAMAGE.
"""
@nb.jit
def nb_dot(x, y):
    val = 0
    for x_i, y_i in zip(x, y):
        val += x_i * y_i
    return val

@nb.jit
def nb_cross(x, y):
    val = np.array([  x[1]*y[2] - x[2]*y[1],
             x[2]*y[0] - x[0]*y[2],
             x[0]*y[1] - x[1]*y[0] ])
    return val

@nb.jit
def r2_circumsphere_tetrahedron_single(a, b, c, d):
    ad = a - d
    bd = b - d
    cd = c - d

    ad2 = nb_dot(ad, ad)
    bd2 = nb_dot(bd, bd)
    cd2 = nb_dot(cd, cd)

    cross_1 = nb_cross(bd, cd)
    cross_2 = nb_cross(cd, ad)
    cross_3 = nb_cross(ad, bd)

    q = ad2 * cross_1 + bd2 * cross_2 + cd2 * cross_3
    p = 2 * np.abs( nb_dot(ad, cross_1) )
    if p < 1e-10:
        return np.infty
    
    r2 = nb_dot(q, q) / p**2

    return r2

@nb.jit(nopython=True)
def r2_circumsphere_tetrahedron(a, b, c, d):
    len_a = len(a)
    r2 = np.zeros((len_a,))
    for i in range(len_a):
        r2[i] = r2_circumsphere_tetrahedron_single(a[i], b[i], c[i], d[i])
    return r2

@nb.jit
def get_faces(tetrahedron):
    faces = np.zeros((4, 3))
    for n, (i1, i2, i3) in enumerate([(0, 1, 2), (0, 1, 3), (0, 2, 3), (1, 2, 3)]):
        faces[n] = tetrahedron[i1], tetrahedron[i2], tetrahedron[i3]
    return faces

def get_single_faces(triangulation):
    num_faces_single = 4
    num_tetrahedrons = triangulation.shape[0]
    num_faces = num_tetrahedrons * num_faces_single
    faces = np.zeros((num_faces, 3), np.int_) # 3 is the dimension of the model
    mask = np.ones((num_faces,), np.bool_)
    for n in range(num_tetrahedrons):
        faces[num_faces_single * n: num_faces_single * (n+1)] = get_faces(triangulation[n])
    orderlist = ["x{}".format(i) for i in range(faces.shape[1])]
    dtype_list = [(el, faces.dtype.str) for el in orderlist]
    faces.view(dtype_list).sort(axis=0)
    for k in range(num_faces-1):
        if mask[k]:
            if np.all(faces[k] == faces[k+1]):
                mask[k] = False
                mask[k+1] = False
    single_faces = faces[mask]
    return single_faces

def get_alpha_shape(pointcloud, alpha):

    pointcloud = np.asarray(pointcloud)
    assert pointcloud.ndim == 2
    assert pointcloud.shape[1] == 3, "for now, only 3-dimensional analysis is implemented"

    triangulation = spat.Delaunay(pointcloud)

    tetrahedrons = pointcloud[triangulation.simplices] # remove this copy step, could be fatal
    radii2 = r2_circumsphere_tetrahedron(tetrahedrons[:, 0, :], tetrahedrons[:, 1, :], tetrahedrons[:, 2, :], tetrahedrons[:, 3, :])
    reduced_triangulation = triangulation.simplices[radii2 < alpha**2]
    del radii2, triangulation, tetrahedrons

    outer_triangulation = get_single_faces(reduced_triangulation)

    return outer_triangulation

In [4]:
# function for reading point cloud from txt files
def array_cloud (filename): 
    f = open(filename,'r')
    point = f.read() #read all the point data from selected file
    f.close()
    l = point.split('\n') 
    l.pop()
    m0 = np.array(l)
    m1 = np.empty((len(l),3))
    for item in range(len(l)):
        n = np.array(m0[item].split(' '))
        m1[item] = n
    return m1

In [5]:
#function for plotting point cloud
def print_cloud (points # the point cloud arary read by above function
                ): 
    x,y,z= np.hsplit(points,3)
    fig=plt.figure(dpi=120)
    ax=fig.add_subplot(111,projection='3d')
    plt.title('point cloud')
    ax.scatter(x,y,z,c='b',marker='.',s=2,linewidth=0,alpha=1,cmap='spectral')
    ax.axis('scaled')          
    ax.set_xlabel('X Label')
    ax.set_ylabel('Y Label')
    ax.set_zlabel('Z Label')
    plt.show()

In [6]:
#collect the point information from the alpha shape faces
def alpha_point(original_cloud, #the original point cloud read from txt file
                alpha_tri #the array list reture by 'get_alpha_shape' function
               ):
    point_set = set()
    
    for item in alpha_tri:
        a = item[0]
        b = item[1]
        c = item[2]
        point_set.add(a)
        point_set.add(b)
        point_set.add(c)
        
    point_list = list(point_set)
    
    alpha_point_array = np.empty((len(point_list),3))
    
    for index in range (len(point_list)):
        n = np.array(original_cloud[point_list[index]])
        alpha_point_array[index] = n
        
    del point_set, point_list
    
    return alpha_point_array #return an array records point information

#direct extract the alpha shape point cloud from original print cloud with determined alpha value
def get_alpha_point (cloud,alpha):
    tri_cl = get_alpha_shape(cloud,alpha)
    array = alpha_point(cloud,tri_cl)
    del tri_cl
    return array

In [7]:
#function for writting point cloud into txt files with ','
def write_cloud(cloud #point cloud in array form
            , filename):
    with open(filename+'.txt','w') as fw:
        for coo in cloud:
            a = str(coo[0])
            b = str(coo[1])
            c = str(coo[2])
            context = a+","+b+','+c+'\n'
            fw.write(context)
        file.close 
    print filename + 'has been written.'

In [8]:
# Function of least square circle fitting by lorenzoriano https://gist.github.com/lorenzoriano/6799568

def calc_R(x,y, xc, yc):
    """ calculate the distance of each 2D points from the center (xc, yc) """
    return np.sqrt((x-xc)**2 + (y-yc)**2)

def f(c, x, y):
    """ calculate the algebraic distance between the data points and the mean circle centered at c=(xc, yc) """
    Ri = calc_R(x, y, *c)
    return Ri - Ri.mean()

def leastsq_circle(x,y):
    # coordinates of the barycenter
    x_m = np.mean(x)
    y_m = np.mean(y)
    center_estimate = x_m, y_m
    center, ier = optimize.leastsq(f, center_estimate, args=(x,y))
    xc, yc = center
    Ri       = calc_R(x, y, *center)
    R        = Ri.mean()
    residu   = np.sum((Ri - R)**2)
    return xc, yc, R, residu

def plot_data_circle(x,y, xc, yc, R):
    f = plt.figure( facecolor='white')  #figsize=(7, 5.4), dpi=72,
    plt.axis('equal')

    theta_fit = np.linspace(-pi, pi, 180)


    plt.xlabel('x')
    plt.ylabel('y')   
    # plot data
    plt.scatter(x, y, c = 'red',label='data')
    x_fit = xc + R*np.cos(theta_fit)
    y_fit = yc + R*np.sin(theta_fit)
    plt.plot(x_fit, y_fit, 'b-' , label="fitted circle", lw=2)
    plt.plot([xc], [yc], 'bD', mec='y', mew=1)

    plt.legend(loc='best',labelspacing=0.1 )
    plt.grid()
    plt.title('Least Squares Circle')

In [9]:
# estimate the DBH of a tree
def DBH(pointcloud #the point cloud from txt file
       ):
    z = pointcloud[:,2]
    height = np.max(z)-np.min(z)
    ground = np.min(z)
    DBHup = ground + 1.35
    DBHdown = ground + 1.25
    DBH_point_list = []
    for item in pointcloud:
        if item[2]>DBHdown and item[2]<DBHup:
            DBH_point_list.append([item[0],item[1]])
    DBH_point_array = np.array(DBH_point_list)
    circle_paras = leastsq_circle(DBH_point_array[:,0],DBH_point_array[:,1])
    val = 2 * circle_paras[2]
    return val

In [10]:
# estimate the trunk height

def trunk_height(pointcloud #the point array from txt file
                ):
    DBH_pointcloud = DBH(pointcloud)
    height = np.max(pointcloud[:,2])-np.min(pointcloud[:,2])
    ground = np.min(pointcloud[:,2])
    n = 0
    level_list = []
    Rlist = []
    for level in range(0,50): #divided the tree into 50 levels with the same height intervals
        x_y_collect = []
        low = ground + n * (height/50)
        up = ground+ (n+1) * (height/50)
        
        for point in pointcloud:
            if point[2] > low and point[2] < up:
                x_y_collect.append([point[0],point[1]])
        x_y_array = np.array(x_y_collect)
        circle_paras = leastsq_circle (x_y_array[:,0],x_y_array[:,1])
        Rlist.append(circle_paras[2])
        if n>1:
            diff = Rlist[-1]-Rlist[-2]
            if diff > 0.25*DBH_pointcloud:
                level_list.append(n)
        n = n + 1
    z = ground + level_list[0] * (height/50)
    return z

In [11]:
# separate the trunk and crown according to the trunk height 
def sep_trunk(pointcloud #point cloud array from txt file directly
              ,alpha_pointcloud #the point cloud obtained from 'alpha_point' function
              ,trunk_name
              ,crown_name):
    height_trunk = trunk_height(pointcloud)
    trunk = pointcloud[np.where(pointcloud[:,2]<=height_trunk)]
    crown = alpha_pointcloud[np.where(alpha_pointcloud[:,2]>height_trunk)]
    write_cloud(trunk,trunk_name)
    write_cloud(crown, crown_name)
    return trunk, crown #returen trunk and crown point cloud array

In [15]:
# define the crown shape according to the least square circle 
# fitting on the upper, middle and lower layer of crown

def crown_shape(crown_cloud): #crown point cloud array)
    z = crown_cloud[:,2]
    crown_height = np.max(z)-np.min(z)
    h = 0.05*crown_height
    crown_down = np.min(z)
    crown_up = np.max(z)
    mid_height = 0.5*crown_height + crown_down
    up = crown_cloud[np.where(crown_cloud[:,2]>=(crown_up-h))]
    m1 = crown_cloud[np.where(crown_cloud[:,2]<=(mid_height+h/2))]
    mid = m1[np.where(m1[:,2]>=(mid_height-h/2))]
    down = crown_cloud[np.where(crown_cloud[:,2]<=(crown_down+h))]
    
    u = leastsq_circle(up[:,0],up[:,1])
    ur = u[2]
    m = leastsq_circle(mid[:,0],mid[:,1])
    mr = m[2]
    d = leastsq_circle(down[:,0],down[:,1])
    dr = d[2]
    
    if dr>mr and mr>ur:
        tag = 'conical'
    elif dr<mr and mr<ur:
        tag = 'inverse conical'
    elif mr>dr and mr>ur:
        tag = 'spherical'
    elif mr<dr and mr<ur:
        tag = 'sandglass'
    else:
        tag = 'cylindrial'
    
    return ur,mr,dr,tag

In [16]:
#function of projection 2D point cloud from 3D point cloud

def project_cloud (cloud): #point cloud array
    x = cloud[:,0]
    y = cloud[:,1]
    xy = np.empty((cloud.size/3, 2))
    xy[:,0] = x
    xy[:,1] = y
    return xy #return 2D point cloud 

In [17]:
#function of estimate the crown projection area and crown width

def crown_width_area(crown_cloud):
    xy = project_cloud(crown_cloud)
    hull = spat.ConvexHull(xy)
    area = hull.area
    point_set = np.empty((hull.vertices.size,2))
    for i in range(hull.vertices.size):
        n = np.array(xy[hull.vertices[i]])
        point_set[i] = n
    R = leastsq_circle(point_set[:,0],point_set[:,1])
    return R[2], area #return largest fitting circle radius and area of 2D crown

In [18]:
# feature data title and fucntion for writting feature data in to txt file
featurename = ['tree','pointnumber','height','alphavalue','pnumber_alpha',
               'trunkheight','crownheight','DBH','crownshape','u','m','d',
               'crownwidth','crownprojectarea','trunkprojectarea','time']

def write_feature (feature, name):
    with open(name+'.txt','w') as fw:
        for coo in feature:
            context = str(coo) + ','
            fw.write(context)
        file.close 
    print name + ' has been written.'
    

In [20]:
#intergrated function for collecting features from tree txt files 

def collectfeature(cloudname): #the txt file address of tree point cloud
    feature = [cloudname] #feature list for append feature data
    trunkname = 'trunk_'+cloudname
    crownname = 'crown_'+cloudname
    cloud = array_cloud (cloudname) #read point cloud as an array 
    height = np.max(cloud[:,2])-np.min(cloud[:,2]) #tree height 
    size = cloud.size/3
    feature.append(size) #record point number
    feature.append(height) #record tree height
    if size > 1000000: #determine alpha value according to point number
        alpha = 2
    elif size < 100000:
        alpha = 0.5
    else:
        alpha = 1
    feature.append(alpha) #record alpha shape
    start = datetime.now() #record start time
    al_cloud = get_alpha_point(cloud,alpha) #extract alpha shape point cloud
    al_name = 'alpha_'+cloudname
    write_cloud(al_cloud,al_name) #write alpha shape point cloud into txt file 
    end = datetime.now() #record alpha shape method end time
    al_size = al_cloud.size/3 #calculate alpha shape point number
    feature.append(al_size) #record above point number
    SEP = sep_trunk(cloud,al_cloud,trunkname,crownname) #seperate trunk and crown 
    trunk = SEP[0] #trunk cloud 
    crown = SEP[1] #crown cloud
    trunkheight = np.max(trunk[:,2])-np.min(trunk[:,2]) #trunk height
    crownheight = np.max(crown[:,2])-np.min(crown[:,2]) #crown height
    feature.append(trunkheight) #record trunk height
    feature.append(crownheight) #record crown height
    dbh = DBH(cloud) #estimate DBH
    feature.append(dbh) #record DBH
    cr_shape = crown_shape(crown) #define the crown shape
    feature.append(cr_shape[-1])
    feature.append(cr_shape[0])
    feature.append(cr_shape[1])
    feature.append(cr_shape[2]) # record crown shape and diameters of each layer
    cw = crown_width(crown) #estimate crown width and crown projection area
    feature.append(cw[0]*2) #record crown width
    feature.append(cw[-1]) #record crown projection area
    trunkarea = trunk_project_area(trunk) #estimate trunk project area
    feature.append(trunkarea) #record trunk projection area
    period = (end-start) #calculate alpha shape time consumption
    feature.append(timedelta.total_seconds(period)) #record the time consumption in second
    feature_name = 'feature_'+ cloudname
    write_feature(feature,feature_name) #write the featrue data into txt file
    return feature